In [2]:
! pip -qq install homoglyphs fonttools

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pytesseract
from PIL import Image,ImageDraw, ImageFont
import pickle
import homoglyphs as hg
from tqdm import tqdm

import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from fontTools.ttLib import TTFont
from fontTools.unicode import Unicode

# font = TTFont('/path/to/font.ttf')

def has_glyph(font, glyph):
    for table in font['cmap'].tables:
        if ord(glyph) in table.cmap.keys():
            return True
    return False



#test
has_glyph(TTFont('../input/unifonts-other/unifont_upper-15.0.01.ttf'),'𝐀')

True

In [5]:
with open(r'../input/char-to-homoglyph-mapping/homoglyphs/mapped_homoglyphs.pkl', 'rb') as f:
    alpha_hg = pickle.load(f)

# CNN

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
import torchvision
from torchvision import transforms, utils, datasets, models
import torch.optim as optim

# from torchsummary import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
class EMNIST_CNN(nn.Module):
    def __init__(self, fmaps1, fmaps2, dense, dropout, num_classes=52):
        super(EMNIST_CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(in_channels=1, out_channels=fmaps1, kernel_size=5, stride=1, padding='same'),                              
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(in_channels=fmaps1, out_channels=fmaps2, kernel_size=5, stride=1, padding='same'),                              
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.fcon1 = nn.Sequential(nn.Linear(49*fmaps2, dense), nn.LeakyReLU())
        self.fcon2 = nn.Linear(dense, num_classes)
        self.dropout = nn.Dropout(p=dropout)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.fcon1(x))
        x = self.fcon2(x)
        return x

In [8]:
idx_char = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'a', 27: 'b', 28: 'c', 29: 'd', 30: 'e', 31: 'f', 32: 'g', 33: 'h', 34: 'i', 35: 'j', 36: 'k', 37: 'l', 38: 'm', 39: 'n', 40: 'o', 41: 'p', 42: 'q', 43: 'r', 44: 's', 45: 't', 46: 'u', 47: 'v', 48: 'w', 49: 'x', 50: 'y', 51: 'z'}

# Character to font mapping

checking if any character doesn't have a font

In [9]:
path ="../input"
#we shall store all the .ttf file names in this list
fontlist = []

for root, dirs, files in os.walk(path):
    for file in files:
        #append the file name to the list
        if file[-4:]=='.ttf':
            fontlist.append(os.path.join(root,file))
    
fontlist

['../input/unifonts-other/unifont_csur-15.0.01.ttf',
 '../input/unifonts-other/unifont_upper-15.0.01.ttf',
 '../input/unifonts-other/unifont_jp-15.0.01.ttf',
 '../input/monospace/monospace.medium.ttf',
 '../input/noto-serif/NotoSerif-Italic.ttf',
 '../input/noto-serif/NotoSerif-BoldItalic.ttf',
 '../input/noto-serif/NotoSerif-Regular.ttf',
 '../input/noto-serif/NotoSerif-Bold.ttf',
 '../input/coptic-eye/Coptic Eyes Latin.ttf',
 '../input/google-noto-sans/NotoSans-Thin.ttf',
 '../input/google-noto-sans/NotoSans-Medium.ttf',
 '../input/google-noto-sans/NotoSans-MediumItalic.ttf',
 '../input/google-noto-sans/NotoSans-LightItalic.ttf',
 '../input/google-noto-sans/NotoSans-ExtraLight.ttf',
 '../input/google-noto-sans/NotoSans-Black.ttf',
 '../input/google-noto-sans/NotoSans-ThinItalic.ttf',
 '../input/google-noto-sans/NotoSans-BoldItalic.ttf',
 '../input/google-noto-sans/NotoSans-Regular.ttf',
 '../input/google-noto-sans/NotoSans-ExtraLightItalic.ttf',
 '../input/google-noto-sans/NotoSans-S

In [10]:
fonts_path = ['../input/unifont/unifont-15.0.01.ttf',
 '../input/ms-gothic/MS Gothic.ttf',
 '../input/google-noto-sans/NotoSans-Thin.ttf',
 '../input/google-noto-sans/NotoSans-Medium.ttf',
 '../input/google-noto-sans/NotoSans-MediumItalic.ttf',
 '../input/google-noto-sans/NotoSans-LightItalic.ttf',
 '../input/google-noto-sans/NotoSans-ExtraLight.ttf',
 '../input/google-noto-sans/NotoSans-Black.ttf',
 '../input/google-noto-sans/NotoSans-ThinItalic.ttf',
 '../input/google-noto-sans/NotoSans-BoldItalic.ttf',
 '../input/google-noto-sans/NotoSans-Regular.ttf',
 '../input/google-noto-sans/NotoSans-ExtraLightItalic.ttf',
 '../input/google-noto-sans/NotoSans-SemiBoldItalic.ttf',
 '../input/google-noto-sans/NotoSans-Light.ttf',
 '../input/google-noto-sans/NotoSans-ExtraBoldItalic.ttf',
 '../input/google-noto-sans/NotoSans-ExtraBold.ttf',
 '../input/google-noto-sans/NotoSans-Bold.ttf',
 '../input/google-noto-sans/NotoSans-BlackItalic.ttf',
 '../input/google-noto-sans/NotoSans-Italic.ttf',
 '../input/google-noto-sans/NotoSans-SemiBold.ttf',
 '../input/times-new-roman/Times New Roman/times new roman italic.ttf',
 '../input/times-new-roman/Times New Roman/times new roman bold italic.ttf',
 '../input/times-new-roman/Times New Roman/times new roman.ttf',
 '../input/times-new-roman/Times New Roman/times new roman bold.ttf',
 '../input/arialttf-font/arial.ttf',
 '../input/symbols/Symbolzm.ttf',
 '../input/coptic-eye/Coptic Eyes Latin.ttf',
 '../input/unifonts-other/unifont_csur-15.0.01.ttf',
 '../input/unifonts-other/unifont_upper-15.0.01.ttf',
 '../input/unifonts-other/unifont_jp-15.0.01.ttf',
 '../input/monospace/monospace.medium.ttf',
 '../input/noto-serif/NotoSerif-Italic.ttf',
 '../input/noto-serif/NotoSerif-BoldItalic.ttf',
 '../input/noto-serif/NotoSerif-Regular.ttf',
 '../input/noto-serif/NotoSerif-Bold.ttf']



In [11]:
! mkdir './all_fonts'

In [12]:
'../input/noto-serif/NotoSerif-Bold.ttf'.split('/')[-1]

'NotoSerif-Bold.ttf'

In [13]:
from shutil import copyfile


for i in fonts_path:
    copyfile(i,os.path.join(r'/kaggle/working/all_fonts',i.split('/')[-1]))

In [ ]:
# fontlist = [
#     '../input/unifonts-other/unifont_csur-15.0.01.ttf',
#  '../input/unifonts-other/unifont_upper-15.0.01.ttf',
#  '../input/unifonts-other/unifont_jp-15.0.01.ttf',
#  '../input/times-new-roman/Times New Roman/times new roman italic.ttf',
#  '../input/times-new-roman/Times New Roman/times new roman bold italic.ttf',
#  '../input/times-new-roman/Times New Roman/times new roman.ttf',
#  '../input/times-new-roman/Times New Roman/times new roman bold.ttf',
#  '../input/ms-gothic/MS Gothic.ttf',
#  '../input/symbols/Symbolzm.ttf',
#  '../input/noto-serif/NotoSerif-Italic.ttf',
#  '../input/noto-serif/NotoSerif-BoldItalic.ttf',
#  '../input/noto-serif/NotoSerif-Regular.ttf',
#  '../input/noto-serif/NotoSerif-Bold.ttf',
#  '../input/unifont/unifont-15.0.01.ttf',
#  '../input/google-noto-sans/NotoSans-Thin.ttf',
#  '../input/google-noto-sans/NotoSans-Medium.ttf',
#  '../input/google-noto-sans/NotoSans-MediumItalic.ttf',
#  '../input/google-noto-sans/NotoSans-LightItalic.ttf',
#  '../input/google-noto-sans/NotoSans-ExtraLight.ttf',
#  '../input/google-noto-sans/NotoSans-Black.ttf',
#  '../input/google-noto-sans/NotoSans-ThinItalic.ttf',
#  '../input/google-noto-sans/NotoSans-BoldItalic.ttf',
#  '../input/google-noto-sans/NotoSans-Regular.ttf',
#  '../input/google-noto-sans/NotoSans-ExtraLightItalic.ttf',
#  '../input/google-noto-sans/NotoSans-SemiBoldItalic.ttf',
#  '../input/google-noto-sans/NotoSans-Light.ttf',
#  '../input/google-noto-sans/NotoSans-ExtraBoldItalic.ttf',
#  '../input/google-noto-sans/NotoSans-ExtraBold.ttf',
#  '../input/google-noto-sans/NotoSans-Bold.ttf',
#  '../input/google-noto-sans/NotoSans-BlackItalic.ttf',
#  '../input/google-noto-sans/NotoSans-Italic.ttf',
#  '../input/google-noto-sans/NotoSans-SemiBold.ttf',
#  '../input/monospace/monospace.medium.ttf',
#  '../input/arialttf-font/arial.ttf',
#  '../input/coptic-eye/Coptic Eyes Latin.ttf']

In [14]:
char_font = {'character':[] , 'font':[]}

for key,values in tqdm(alpha_hg.items()):
    for char in values:
        if len(char)==1:
            for font in fontlist:
                if has_glyph(TTFont(font),char):
                    char_font['character'].append(char)
                    char_font['font'].append(font)
#                     char_font['font'].append(font[9:])
                    break
            if char_font['character'][-1] != char:
                char_font['character'].append(char)
                char_font['font'].append('NO FONT!!')

100%|██████████| 52/52 [01:16<00:00,  1.47s/it]


In [15]:
font_df = pd.DataFrame(char_font)
font_df

,character,font
0,A,../input/unifonts-other/unifont_jp-15.0.01.ttf
1,Ａ,../input/unifonts-other/unifont_jp-15.0.01.ttf
2,𝐀,../input/unifonts-other/unifont_upper-15.0.01.ttf
3,𝐴,../input/unifonts-other/unifont_upper-15.0.01.ttf
4,𝑨,../input/unifonts-other/unifont_upper-15.0.01.ttf
...,...,...
1008,𝗓,../input/unifonts-other/unifont_upper-15.0.01.ttf
1009,𝘇,../input/unifonts-other/unifont_upper-15.0.01.ttf
1010,𝘻,../input/unifonts-other/unifont_upper-15.0.01.ttf
1011,𝙯,../input/unifonts-other/unifont_upper-15.0.01.ttf


In [ ]:
# font_df.to_csv('char_font.csv')

In [16]:
font_df[font_df['font']=='NO FONT!!']

,character,font


In [17]:
font_df['font'].value_counts()

../input/unifonts-other/unifont_upper-15.0.01.ttf    803
../input/unifonts-other/unifont_jp-15.0.01.ttf       210
Name: font, dtype: int64

In [18]:
char_font_direct = {i:j for i,j in zip(char_font['character'],char_font['font'])}
# char_font_direct

# rendering chars to image and testing them with tesseract OCR

In [19]:
! rm -R './tesseract_ocr_results'
! mkdir './tesseract_ocr_results'

rm: cannot remove './tesseract_ocr_results': No such file or directory


In [20]:
for i,c in idx_char.items():
    os.mkdir(os.path.join(r'./tesseract_ocr_results',c))

## tesseract

In [21]:
total = 0
correct, correct_uncased = 0, 0
cat=set()
all_chars = ''

for key,values in tqdm(alpha_hg.items()):
    for char in values:
        
        all_chars = all_chars+ char
        
        total+=1
        
        if len(char)==1:
            cat.add(hg.Categories.detect(char))
        
        # Create black mask using Numpy and convert from BGR (OpenCV) to RGB (PIL)
#         image = np.zeros((100, 100, 3), dtype=np.uint8)
        image = np.zeros((28, 28, 3), dtype=np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(image)

        # Draw non-ascii text onto image
        spacing=0
#         for single_char in char:
#             font = ImageFont.truetype(char_font_direct[single_char], 35)
#             draw = ImageDraw.Draw(pil_image)
#             draw.text((30+spacing, 30), single_char, font=font)
#             spacing+=10
        for single_char in char:
            font = ImageFont.truetype(char_font_direct[single_char], 25)
            draw = ImageDraw.Draw(pil_image)
            draw.text((spacing, 0), single_char, font=font)
            spacing+=10
        
        # Convert back to Numpy array and switch back from RGB to BGR
        img= np.asarray(pil_image)
        img= cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        # from https://stackoverflow.com/a/50762612/10997981
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        kernel = np.ones((2, 2), np.uint8)
        img = cv2.dilate(img, kernel, iterations=1)
        # img = cv2.erode(img, kernel, iterations=1)

        img = cv2.threshold(cv2.medianBlur(img, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        
        text = pytesseract.image_to_string(img, config = "--psm 10 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")[:-2]
        
        pil_image.save(f"./tesseract_ocr_results/{key}/{key}__char_{char}__pred_{text}__orig.png")
        cv2.imwrite(f"./tesseract_ocr_results/{key}/{key}__char_{char}__pred_{text}__processed.png", img)
        
        if text.upper() == key or text.lower() == key: #by_merge or uncased
            correct_uncased+=1
        
        if text == key:
            correct+=1
        
        

100%|██████████| 52/52 [09:04<00:00, 10.47s/it]


## CNN

In [22]:
total = 0
correct, correct_uncased = 0, 0
cat=set()
all_chars = ''

model = EMNIST_CNN(40, 160, 200, 0.4, 52).to(device)


checkpoint_path = r'../input/hemnistcheckpoint/HEMNIST_CNN_7.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

transform_tensor = transforms.ToTensor()

for key,values in tqdm(alpha_hg.items()):
#     print(values)
    for char in values:
        
        all_chars = all_chars+ char
        
        total+=1
#         print(char, '\t', total)

        
        if len(char)==1:
            cat.add(hg.Categories.detect(char))
        
        # Create black mask using Numpy and convert from BGR (OpenCV) to RGB (PIL)
        # image = cv2.imread('1.png') # If you were using an actual image
        image = np.zeros((28, 28, 3), dtype=np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(image)

        # Draw non-ascii text onto image
        spacing=0
        for single_char in char:
            font = ImageFont.truetype(char_font_direct[single_char], 25)
            draw = ImageDraw.Draw(pil_image)
            draw.text((spacing, 0), single_char, font=font)
            spacing+=10


        # Convert back to Numpy array and switch back from RGB to BGR
        img= np.asarray(pil_image)
        img= cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        kernel = np.ones((2, 2), np.uint8)
        img = cv2.dilate(img, kernel, iterations=1)
#         img = cv2.erode(img, kernel, iterations=1)

#         img = cv2.threshold(cv2.medianBlur(img, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

        img = Image.fromarray(img)
        
        inp_img = transform_tensor(img)
        inp_img = inp_img.unsqueeze(-1).permute(0, 3, 1, 2)
        
        text = idx_char[torch.argmax(model(inp_img)).item()]
        
        img.save(f"./tesseract_ocr_results/{key}/{key}__char_{char}__pred_{text}__processed.png")
        pil_image.save(f"./tesseract_ocr_results/{key}/{key}__char_{char}__pred_{text}__original.png")
        
        if text.upper() == key or text.lower() == key: #by_merge
            correct_uncased+=1
        
        if text == key: # by_class
            correct+=1
#         else:
#             print(f'WRONG for {key} : {char} : {text}')
        
        

100%|██████████| 52/52 [00:19<00:00,  2.61it/s]


## Metrics

In [23]:
print(correct/total*100)

61.533586818757925


In [24]:
print(correct_uncased/total*100)

63.05449936628644


In [25]:
print(f'CORRECT={correct} TOTAL={total}')

CORRECT=1942 TOTAL=3156


In [26]:
print(f'CORRECT={correct_uncased} TOTAL={total}')

CORRECT=1990 TOTAL=3156


In [27]:
cat # {'COMMON' : '../input/arialttf-font', 'COPTIC': '../input/ms-gothic', 'LATIN':'../input/arialttf-font'}

{'COMMON', 'COPTIC', 'LATIN'}

In [ ]:
# all_chars

# Exploring output

In [25]:
# [os.listdir(os.path.join('/kaggle/working/tesseract_ocr_results',j)) for j in [i for i in os.listdir('/kaggle/working/tesseract_ocr_results')]]
all=[]

for i in os.listdir('/kaggle/working/tesseract_ocr_results'):
    for j in os.listdir(os.path.join('/kaggle/working/tesseract_ocr_results',i)):
        all.append(j)
        
len(all)

6312

# OCR test n stuff(ROUGH)

In [ ]:
s2 = 'AＡ𝐀𝐴𝑨𝒜𝓐𝔄𝔸𝕬𝖠𝗔𝘈𝘼𝙰𝚨𝛢𝜜𝝖𝞐BℬꞴＢ𝐁𝐵𝑩𝓑𝔅𝔹𝕭𝖡𝗕𝘉𝘽𝙱𝚩𝛣𝜝𝝗𝞑CℂℭⅭＣ𝐂𝐶𝑪𝒞𝓒𝕮𝖢𝗖𝘊𝘾𝙲🝌DⅅⅮ𝐃𝐷𝑫𝒟𝓓𝔇𝔻𝕯𝖣𝗗𝘋𝘿𝙳Eℰ⋿Ｅ𝐄𝐸𝑬𝓔𝔈𝔼𝕰𝖤𝗘𝘌𝙀𝙴𝚬𝛦𝜠𝝚𝞔FℱꞘ𝐅𝐹𝑭𝓕𝔉𝔽𝕱𝖥𝗙𝘍𝙁𝙵𝟊G𝐆𝐺𝑮𝒢𝓖𝔊𝔾𝕲𝖦𝗚𝘎𝙂𝙶HℋℌℍＨ𝐇𝐻𝑯𝓗𝕳𝖧𝗛𝘏𝙃𝙷𝚮𝛨𝜢𝝜𝞖1Il|ƖǀℐℑℓⅠⅼ∣⏽Ｉｌ￨𝐈𝐥𝐼𝑙𝑰𝒍𝓁𝓘𝓵𝔩𝕀𝕝𝕴𝖑𝖨𝗅𝗜𝗹𝘐𝘭𝙄𝙡𝙸𝚕𝚰𝛪𝜤𝝞𝞘𝟏𝟙𝟣𝟭𝟷JꞲＪ𝐉𝐽𝑱𝒥𝓙𝔍𝕁𝕵𝖩𝗝𝘑𝙅𝙹KKＫ𝐊𝐾𝑲𝒦𝓚𝔎𝕂𝕶𝖪𝗞𝘒𝙆𝙺𝚱𝛫𝜥𝝟𝞙LℒⅬ𝐋𝐿𝑳𝓛𝔏𝕃𝕷𝖫𝗟𝘓𝙇𝙻MℳⅯＭ𝐌𝑀𝑴𝓜𝔐𝕄𝕸𝖬𝗠𝘔𝙈𝙼𝚳𝛭𝜧𝝡𝞛NℕＮ𝐍𝑁𝑵𝒩𝓝𝔑𝕹𝖭𝗡𝘕𝙉𝙽𝚴𝛮𝜨𝝢𝞜0OＯ𝐎𝑂𝑶𝒪𝓞𝔒𝕆𝕺𝖮𝗢𝘖𝙊𝙾𝚶𝛰𝜪𝝤𝞞𝟎𝟘𝟢𝟬𝟶PℙＰ𝐏𝑃𝑷𝒫𝓟𝔓𝕻𝖯𝗣𝘗𝙋𝙿𝚸𝛲𝜬𝝦𝞠Qℚ𝐐𝑄𝑸𝒬𝓠𝔔𝕼𝖰𝗤𝘘𝙌𝚀RƦℛℜℝ𝐑𝑅𝑹𝓡𝕽𝖱𝗥𝘙𝙍𝚁SＳ𝐒𝑆𝑺𝒮𝓢𝔖𝕊𝕾𝖲𝗦𝘚𝙎𝚂T⊤⟙Ｔ𝐓𝑇𝑻𝒯𝓣𝔗𝕋𝕿𝖳𝗧𝘛𝙏𝚃𝚻𝛵𝜯𝝩𝞣🝨U∪⋃𝐔𝑈𝑼𝒰𝓤𝔘𝕌𝖀𝖴𝗨𝘜𝙐𝚄VⅤ𝐕𝑉𝑽𝒱𝓥𝔙𝕍𝖁𝖵𝗩𝘝𝙑𝚅W𝐖𝑊𝑾𝒲𝓦𝔚𝕎𝖂𝖶𝗪𝘞𝙒𝚆XⅩ╳ꞳＸ𝐗𝑋𝑿𝒳𝓧𝔛𝕏𝖃𝖷𝗫𝘟𝙓𝚇𝚾𝛸𝜲𝝬𝞦YＹ𝐘𝑌𝒀𝒴𝓨𝔜𝕐𝖄𝖸𝗬𝘠𝙔𝚈𝚼𝛶𝜰𝝪𝞤ZℤℨＺ𐋵𝐙𝑍𝒁𝒵𝓩𝖅𝖹𝗭𝘡𝙕𝚉𝚭𝛧𝜡𝝛𝞕aɑ⍺ａ𝐚𝑎𝒂𝒶𝓪𝔞𝕒𝖆𝖺𝗮𝘢𝙖𝚊𝛂𝛼𝜶𝝰𝞪cicıcɩcɪc˛cℹcⅈcⅰc⍳cｉc𝐢c𝑖c𝒊c𝒾c𝓲c𝔦c𝕚c𝖎c𝗂c𝗶c𝘪c𝙞c𝚒c𝚤c𝛊c𝜄c𝜾c𝝸c𝞲ᴄiᴄıᴄɩᴄɪᴄ˛ᴄℹᴄⅈᴄⅰᴄ⍳ᴄｉᴄ𝐢ᴄ𝑖ᴄ𝒊ᴄ𝒾ᴄ𝓲ᴄ𝔦ᴄ𝕚ᴄ𝖎ᴄ𝗂ᴄ𝗶ᴄ𝘪ᴄ𝙞ᴄ𝚒ᴄ𝚤ᴄ𝛊ᴄ𝜄ᴄ𝜾ᴄ𝝸ᴄ𝞲ⅽiⅽıⅽɩⅽɪⅽ˛ⅽℹⅽⅈⅽⅰⅽ⍳ⅽｉⅽ𝐢ⅽ𝑖ⅽ𝒊ⅽ𝒾ⅽ𝓲ⅽ𝔦ⅽ𝕚ⅽ𝖎ⅽ𝗂ⅽ𝗶ⅽ𝘪ⅽ𝙞ⅽ𝚒ⅽ𝚤ⅽ𝛊ⅽ𝜄ⅽ𝜾ⅽ𝝸ⅽ𝞲ｃiｃıｃɩｃɪｃ˛ｃℹｃⅈｃⅰｃ⍳ｃｉｃ𝐢ｃ𝑖ｃ𝒊ｃ𝒾ｃ𝓲ｃ𝔦ｃ𝕚ｃ𝖎ｃ𝗂ｃ𝗶ｃ𝘪ｃ𝙞ｃ𝚒ｃ𝚤ｃ𝛊ｃ𝜄ｃ𝜾ｃ𝝸ｃ𝞲𝐜i𝐜ı𝐜ɩ𝐜ɪ𝐜˛𝐜ℹ𝐜ⅈ𝐜ⅰ𝐜⍳𝐜ｉ𝐜𝐢𝐜𝑖𝐜𝒊𝐜𝒾𝐜𝓲𝐜𝔦𝐜𝕚𝐜𝖎𝐜𝗂𝐜𝗶𝐜𝘪𝐜𝙞𝐜𝚒𝐜𝚤𝐜𝛊𝐜𝜄𝐜𝜾𝐜𝝸𝐜𝞲𝑐i𝑐ı𝑐ɩ𝑐ɪ𝑐˛𝑐ℹ𝑐ⅈ𝑐ⅰ𝑐⍳𝑐ｉ𝑐𝐢𝑐𝑖𝑐𝒊𝑐𝒾𝑐𝓲𝑐𝔦𝑐𝕚𝑐𝖎𝑐𝗂𝑐𝗶𝑐𝘪𝑐𝙞𝑐𝚒𝑐𝚤𝑐𝛊𝑐𝜄𝑐𝜾𝑐𝝸𝑐𝞲𝒄i𝒄ı𝒄ɩ𝒄ɪ𝒄˛𝒄ℹ𝒄ⅈ𝒄ⅰ𝒄⍳𝒄ｉ𝒄𝐢𝒄𝑖𝒄𝒊𝒄𝒾𝒄𝓲𝒄𝔦𝒄𝕚𝒄𝖎𝒄𝗂𝒄𝗶𝒄𝘪𝒄𝙞𝒄𝚒𝒄𝚤𝒄𝛊𝒄𝜄𝒄𝜾𝒄𝝸𝒄𝞲𝒸i𝒸ı𝒸ɩ𝒸ɪ𝒸˛𝒸ℹ𝒸ⅈ𝒸ⅰ𝒸⍳𝒸ｉ𝒸𝐢𝒸𝑖𝒸𝒊𝒸𝒾𝒸𝓲𝒸𝔦𝒸𝕚𝒸𝖎𝒸𝗂𝒸𝗶𝒸𝘪𝒸𝙞𝒸𝚒𝒸𝚤𝒸𝛊𝒸𝜄𝒸𝜾𝒸𝝸𝒸𝞲𝓬i𝓬ı𝓬ɩ𝓬ɪ𝓬˛𝓬ℹ𝓬ⅈ𝓬ⅰ𝓬⍳𝓬ｉ𝓬𝐢𝓬𝑖𝓬𝒊𝓬𝒾𝓬𝓲𝓬𝔦𝓬𝕚𝓬𝖎𝓬𝗂𝓬𝗶𝓬𝘪𝓬𝙞𝓬𝚒𝓬𝚤𝓬𝛊𝓬𝜄𝓬𝜾𝓬𝝸𝓬𝞲𝔠i𝔠ı𝔠ɩ𝔠ɪ𝔠˛𝔠ℹ𝔠ⅈ𝔠ⅰ𝔠⍳𝔠ｉ𝔠𝐢𝔠𝑖𝔠𝒊𝔠𝒾𝔠𝓲𝔠𝔦𝔠𝕚𝔠𝖎𝔠𝗂𝔠𝗶𝔠𝘪𝔠𝙞𝔠𝚒𝔠𝚤𝔠𝛊𝔠𝜄𝔠𝜾𝔠𝝸𝔠𝞲𝕔i𝕔ı𝕔ɩ𝕔ɪ𝕔˛𝕔ℹ𝕔ⅈ𝕔ⅰ𝕔⍳𝕔ｉ𝕔𝐢𝕔𝑖𝕔𝒊𝕔𝒾𝕔𝓲𝕔𝔦𝕔𝕚𝕔𝖎𝕔𝗂𝕔𝗶𝕔𝘪𝕔𝙞𝕔𝚒𝕔𝚤𝕔𝛊𝕔𝜄𝕔𝜾𝕔𝝸𝕔𝞲𝖈i𝖈ı𝖈ɩ𝖈ɪ𝖈˛𝖈ℹ𝖈ⅈ𝖈ⅰ𝖈⍳𝖈ｉ𝖈𝐢𝖈𝑖𝖈𝒊𝖈𝒾𝖈𝓲𝖈𝔦𝖈𝕚𝖈𝖎𝖈𝗂𝖈𝗶𝖈𝘪𝖈𝙞𝖈𝚒𝖈𝚤𝖈𝛊𝖈𝜄𝖈𝜾𝖈𝝸𝖈𝞲𝖼i𝖼ı𝖼ɩ𝖼ɪ𝖼˛𝖼ℹ𝖼ⅈ𝖼ⅰ𝖼⍳𝖼ｉ𝖼𝐢𝖼𝑖𝖼𝒊𝖼𝒾𝖼𝓲𝖼𝔦𝖼𝕚𝖼𝖎𝖼𝗂𝖼𝗶𝖼𝘪𝖼𝙞𝖼𝚒𝖼𝚤𝖼𝛊𝖼𝜄𝖼𝜾𝖼𝝸𝖼𝞲𝗰i𝗰ı𝗰ɩ𝗰ɪ𝗰˛𝗰ℹ𝗰ⅈ𝗰ⅰ𝗰⍳𝗰ｉ𝗰𝐢𝗰𝑖𝗰𝒊𝗰𝒾𝗰𝓲𝗰𝔦𝗰𝕚𝗰𝖎𝗰𝗂𝗰𝗶𝗰𝘪𝗰𝙞𝗰𝚒𝗰𝚤𝗰𝛊𝗰𝜄𝗰𝜾𝗰𝝸𝗰𝞲𝘤i𝘤ı𝘤ɩ𝘤ɪ𝘤˛𝘤ℹ𝘤ⅈ𝘤ⅰ𝘤⍳𝘤ｉ𝘤𝐢𝘤𝑖𝘤𝒊𝘤𝒾𝘤𝓲𝘤𝔦𝘤𝕚𝘤𝖎𝘤𝗂𝘤𝗶𝘤𝘪𝘤𝙞𝘤𝚒𝘤𝚤𝘤𝛊𝘤𝜄𝘤𝜾𝘤𝝸𝘤𝞲𝙘i𝙘ı𝙘ɩ𝙘ɪ𝙘˛𝙘ℹ𝙘ⅈ𝙘ⅰ𝙘⍳𝙘ｉ𝙘𝐢𝙘𝑖𝙘𝒊𝙘𝒾𝙘𝓲𝙘𝔦𝙘𝕚𝙘𝖎𝙘𝗂𝙘𝗶𝙘𝘪𝙘𝙞𝙘𝚒𝙘𝚤𝙘𝛊𝙘𝜄𝙘𝜾𝙘𝝸𝙘𝞲𝚌i𝚌ı𝚌ɩ𝚌ɪ𝚌˛𝚌ℹ𝚌ⅈ𝚌ⅰ𝚌⍳𝚌ｉ𝚌𝐢𝚌𝑖𝚌𝒊𝚌𝒾𝚌𝓲𝚌𝔦𝚌𝕚𝚌𝖎𝚌𝗂𝚌𝗶𝚌𝘪𝚌𝙞𝚌𝚒𝚌𝚤𝚌𝛊𝚌𝜄𝚌𝜾𝚌𝝸𝚌𝞲bƄ𝐛𝑏𝒃𝒷𝓫𝔟𝕓𝖇𝖻𝗯𝘣𝙗𝚋cᴄⅽｃ𝐜𝑐𝒄𝒸𝓬𝔠𝕔𝖈𝖼𝗰𝘤𝙘𝚌dⅆⅾ𝐝𝑑𝒅𝒹𝓭𝔡𝕕𝖉𝖽𝗱𝘥𝙙𝚍c1cIclc|cƖcǀcℐcℑcℓcⅠcⅼc∣c⏽cＩcｌc￨c𝐈c𝐥c𝐼c𝑙c𝑰c𝒍c𝓁c𝓘c𝓵c𝔩c𝕀c𝕝c𝕴c𝖑c𝖨c𝗅c𝗜c𝗹c𝘐c𝘭c𝙄c𝙡c𝙸c𝚕c𝚰c𝛪c𝜤c𝝞c𝞘c𝟏c𝟙c𝟣c𝟭c𝟷ᴄ1ᴄIᴄlᴄ|ᴄƖᴄǀᴄℐᴄℑᴄℓᴄⅠᴄⅼᴄ∣ᴄ⏽ᴄＩᴄｌᴄ￨ᴄ𝐈ᴄ𝐥ᴄ𝐼ᴄ𝑙ᴄ𝑰ᴄ𝒍ᴄ𝓁ᴄ𝓘ᴄ𝓵ᴄ𝔩ᴄ𝕀ᴄ𝕝ᴄ𝕴ᴄ𝖑ᴄ𝖨ᴄ𝗅ᴄ𝗜ᴄ𝗹ᴄ𝘐ᴄ𝘭ᴄ𝙄ᴄ𝙡ᴄ𝙸ᴄ𝚕ᴄ𝚰ᴄ𝛪ᴄ𝜤ᴄ𝝞ᴄ𝞘ᴄ𝟏ᴄ𝟙ᴄ𝟣ᴄ𝟭ᴄ𝟷ⅽ1ⅽIⅽlⅽ|ⅽƖⅽǀⅽℐⅽℑⅽℓⅽⅠⅽⅼⅽ∣ⅽ⏽ⅽＩⅽｌⅽ￨ⅽ𝐈ⅽ𝐥ⅽ𝐼ⅽ𝑙ⅽ𝑰ⅽ𝒍ⅽ𝓁ⅽ𝓘ⅽ𝓵ⅽ𝔩ⅽ𝕀ⅽ𝕝ⅽ𝕴ⅽ𝖑ⅽ𝖨ⅽ𝗅ⅽ𝗜ⅽ𝗹ⅽ𝘐ⅽ𝘭ⅽ𝙄ⅽ𝙡ⅽ𝙸ⅽ𝚕ⅽ𝚰ⅽ𝛪ⅽ𝜤ⅽ𝝞ⅽ𝞘ⅽ𝟏ⅽ𝟙ⅽ𝟣ⅽ𝟭ⅽ𝟷ｃ1ｃIｃlｃ|ｃƖｃǀｃℐｃℑｃℓｃⅠｃⅼｃ∣ｃ⏽ｃＩｃｌｃ￨ｃ𝐈ｃ𝐥ｃ𝐼ｃ𝑙ｃ𝑰ｃ𝒍ｃ𝓁ｃ𝓘ｃ𝓵ｃ𝔩ｃ𝕀ｃ𝕝ｃ𝕴ｃ𝖑ｃ𝖨ｃ𝗅ｃ𝗜ｃ𝗹ｃ𝘐ｃ𝘭ｃ𝙄ｃ𝙡ｃ𝙸ｃ𝚕ｃ𝚰ｃ𝛪ｃ𝜤ｃ𝝞ｃ𝞘ｃ𝟏ｃ𝟙ｃ𝟣ｃ𝟭ｃ𝟷𝐜1𝐜I𝐜l𝐜|𝐜Ɩ𝐜ǀ𝐜ℐ𝐜ℑ𝐜ℓ𝐜Ⅰ𝐜ⅼ𝐜∣𝐜⏽𝐜Ｉ𝐜ｌ𝐜￨𝐜𝐈𝐜𝐥𝐜𝐼𝐜𝑙𝐜𝑰𝐜𝒍𝐜𝓁𝐜𝓘𝐜𝓵𝐜𝔩𝐜𝕀𝐜𝕝𝐜𝕴𝐜𝖑𝐜𝖨𝐜𝗅𝐜𝗜𝐜𝗹𝐜𝘐𝐜𝘭𝐜𝙄𝐜𝙡𝐜𝙸𝐜𝚕𝐜𝚰𝐜𝛪𝐜𝜤𝐜𝝞𝐜𝞘𝐜𝟏𝐜𝟙𝐜𝟣𝐜𝟭𝐜𝟷𝑐1𝑐I𝑐l𝑐|𝑐Ɩ𝑐ǀ𝑐ℐ𝑐ℑ𝑐ℓ𝑐Ⅰ𝑐ⅼ𝑐∣𝑐⏽𝑐Ｉ𝑐ｌ𝑐￨𝑐𝐈𝑐𝐥𝑐𝐼𝑐𝑙𝑐𝑰𝑐𝒍𝑐𝓁𝑐𝓘𝑐𝓵𝑐𝔩𝑐𝕀𝑐𝕝𝑐𝕴𝑐𝖑𝑐𝖨𝑐𝗅𝑐𝗜𝑐𝗹𝑐𝘐𝑐𝘭𝑐𝙄𝑐𝙡𝑐𝙸𝑐𝚕𝑐𝚰𝑐𝛪𝑐𝜤𝑐𝝞𝑐𝞘𝑐𝟏𝑐𝟙𝑐𝟣𝑐𝟭𝑐𝟷𝒄1𝒄I𝒄l𝒄|𝒄Ɩ𝒄ǀ𝒄ℐ𝒄ℑ𝒄ℓ𝒄Ⅰ𝒄ⅼ𝒄∣𝒄⏽𝒄Ｉ𝒄ｌ𝒄￨𝒄𝐈𝒄𝐥𝒄𝐼𝒄𝑙𝒄𝑰𝒄𝒍𝒄𝓁𝒄𝓘𝒄𝓵𝒄𝔩𝒄𝕀𝒄𝕝𝒄𝕴𝒄𝖑𝒄𝖨𝒄𝗅𝒄𝗜𝒄𝗹𝒄𝘐𝒄𝘭𝒄𝙄𝒄𝙡𝒄𝙸𝒄𝚕𝒄𝚰𝒄𝛪𝒄𝜤𝒄𝝞𝒄𝞘𝒄𝟏𝒄𝟙𝒄𝟣𝒄𝟭𝒄𝟷𝒸1𝒸I𝒸l𝒸|𝒸Ɩ𝒸ǀ𝒸ℐ𝒸ℑ𝒸ℓ𝒸Ⅰ𝒸ⅼ𝒸∣𝒸⏽𝒸Ｉ𝒸ｌ𝒸￨𝒸𝐈𝒸𝐥𝒸𝐼𝒸𝑙𝒸𝑰𝒸𝒍𝒸𝓁𝒸𝓘𝒸𝓵𝒸𝔩𝒸𝕀𝒸𝕝𝒸𝕴𝒸𝖑𝒸𝖨𝒸𝗅𝒸𝗜𝒸𝗹𝒸𝘐𝒸𝘭𝒸𝙄𝒸𝙡𝒸𝙸𝒸𝚕𝒸𝚰𝒸𝛪𝒸𝜤𝒸𝝞𝒸𝞘𝒸𝟏𝒸𝟙𝒸𝟣𝒸𝟭𝒸𝟷𝓬1𝓬I𝓬l𝓬|𝓬Ɩ𝓬ǀ𝓬ℐ𝓬ℑ𝓬ℓ𝓬Ⅰ𝓬ⅼ𝓬∣𝓬⏽𝓬Ｉ𝓬ｌ𝓬￨𝓬𝐈𝓬𝐥𝓬𝐼𝓬𝑙𝓬𝑰𝓬𝒍𝓬𝓁𝓬𝓘𝓬𝓵𝓬𝔩𝓬𝕀𝓬𝕝𝓬𝕴𝓬𝖑𝓬𝖨𝓬𝗅𝓬𝗜𝓬𝗹𝓬𝘐𝓬𝘭𝓬𝙄𝓬𝙡𝓬𝙸𝓬𝚕𝓬𝚰𝓬𝛪𝓬𝜤𝓬𝝞𝓬𝞘𝓬𝟏𝓬𝟙𝓬𝟣𝓬𝟭𝓬𝟷𝔠1𝔠I𝔠l𝔠|𝔠Ɩ𝔠ǀ𝔠ℐ𝔠ℑ𝔠ℓ𝔠Ⅰ𝔠ⅼ𝔠∣𝔠⏽𝔠Ｉ𝔠ｌ𝔠￨𝔠𝐈𝔠𝐥𝔠𝐼𝔠𝑙𝔠𝑰𝔠𝒍𝔠𝓁𝔠𝓘𝔠𝓵𝔠𝔩𝔠𝕀𝔠𝕝𝔠𝕴𝔠𝖑𝔠𝖨𝔠𝗅𝔠𝗜𝔠𝗹𝔠𝘐𝔠𝘭𝔠𝙄𝔠𝙡𝔠𝙸𝔠𝚕𝔠𝚰𝔠𝛪𝔠𝜤𝔠𝝞𝔠𝞘𝔠𝟏𝔠𝟙𝔠𝟣𝔠𝟭𝔠𝟷𝕔1𝕔I𝕔l𝕔|𝕔Ɩ𝕔ǀ𝕔ℐ𝕔ℑ𝕔ℓ𝕔Ⅰ𝕔ⅼ𝕔∣𝕔⏽𝕔Ｉ𝕔ｌ𝕔￨𝕔𝐈𝕔𝐥𝕔𝐼𝕔𝑙𝕔𝑰𝕔𝒍𝕔𝓁𝕔𝓘𝕔𝓵𝕔𝔩𝕔𝕀𝕔𝕝𝕔𝕴𝕔𝖑𝕔𝖨𝕔𝗅𝕔𝗜𝕔𝗹𝕔𝘐𝕔𝘭𝕔𝙄𝕔𝙡𝕔𝙸𝕔𝚕𝕔𝚰𝕔𝛪𝕔𝜤𝕔𝝞𝕔𝞘𝕔𝟏𝕔𝟙𝕔𝟣𝕔𝟭𝕔𝟷𝖈1𝖈I𝖈l𝖈|𝖈Ɩ𝖈ǀ𝖈ℐ𝖈ℑ𝖈ℓ𝖈Ⅰ𝖈ⅼ𝖈∣𝖈⏽𝖈Ｉ𝖈ｌ𝖈￨𝖈𝐈𝖈𝐥𝖈𝐼𝖈𝑙𝖈𝑰𝖈𝒍𝖈𝓁𝖈𝓘𝖈𝓵𝖈𝔩𝖈𝕀𝖈𝕝𝖈𝕴𝖈𝖑𝖈𝖨𝖈𝗅𝖈𝗜𝖈𝗹𝖈𝘐𝖈𝘭𝖈𝙄𝖈𝙡𝖈𝙸𝖈𝚕𝖈𝚰𝖈𝛪𝖈𝜤𝖈𝝞𝖈𝞘𝖈𝟏𝖈𝟙𝖈𝟣𝖈𝟭𝖈𝟷𝖼1𝖼I𝖼l𝖼|𝖼Ɩ𝖼ǀ𝖼ℐ𝖼ℑ𝖼ℓ𝖼Ⅰ𝖼ⅼ𝖼∣𝖼⏽𝖼Ｉ𝖼ｌ𝖼￨𝖼𝐈𝖼𝐥𝖼𝐼𝖼𝑙𝖼𝑰𝖼𝒍𝖼𝓁𝖼𝓘𝖼𝓵𝖼𝔩𝖼𝕀𝖼𝕝𝖼𝕴𝖼𝖑𝖼𝖨𝖼𝗅𝖼𝗜𝖼𝗹𝖼𝘐𝖼𝘭𝖼𝙄𝖼𝙡𝖼𝙸𝖼𝚕𝖼𝚰𝖼𝛪𝖼𝜤𝖼𝝞𝖼𝞘𝖼𝟏𝖼𝟙𝖼𝟣𝖼𝟭𝖼𝟷𝗰1𝗰I𝗰l𝗰|𝗰Ɩ𝗰ǀ𝗰ℐ𝗰ℑ𝗰ℓ𝗰Ⅰ𝗰ⅼ𝗰∣𝗰⏽𝗰Ｉ𝗰ｌ𝗰￨𝗰𝐈𝗰𝐥𝗰𝐼𝗰𝑙𝗰𝑰𝗰𝒍𝗰𝓁𝗰𝓘𝗰𝓵𝗰𝔩𝗰𝕀𝗰𝕝𝗰𝕴𝗰𝖑𝗰𝖨𝗰𝗅𝗰𝗜𝗰𝗹𝗰𝘐𝗰𝘭𝗰𝙄𝗰𝙡𝗰𝙸𝗰𝚕𝗰𝚰𝗰𝛪𝗰𝜤𝗰𝝞𝗰𝞘𝗰𝟏𝗰𝟙𝗰𝟣𝗰𝟭𝗰𝟷𝘤1𝘤I𝘤l𝘤|𝘤Ɩ𝘤ǀ𝘤ℐ𝘤ℑ𝘤ℓ𝘤Ⅰ𝘤ⅼ𝘤∣𝘤⏽𝘤Ｉ𝘤ｌ𝘤￨𝘤𝐈𝘤𝐥𝘤𝐼𝘤𝑙𝘤𝑰𝘤𝒍𝘤𝓁𝘤𝓘𝘤𝓵𝘤𝔩𝘤𝕀𝘤𝕝𝘤𝕴𝘤𝖑𝘤𝖨𝘤𝗅𝘤𝗜𝘤𝗹𝘤𝘐𝘤𝘭𝘤𝙄𝘤𝙡𝘤𝙸𝘤𝚕𝘤𝚰𝘤𝛪𝘤𝜤𝘤𝝞𝘤𝞘𝘤𝟏𝘤𝟙𝘤𝟣𝘤𝟭𝘤𝟷𝙘1𝙘I𝙘l𝙘|𝙘Ɩ𝙘ǀ𝙘ℐ𝙘ℑ𝙘ℓ𝙘Ⅰ𝙘ⅼ𝙘∣𝙘⏽𝙘Ｉ𝙘ｌ𝙘￨𝙘𝐈𝙘𝐥𝙘𝐼𝙘𝑙𝙘𝑰𝙘𝒍𝙘𝓁𝙘𝓘𝙘𝓵𝙘𝔩𝙘𝕀𝙘𝕝𝙘𝕴𝙘𝖑𝙘𝖨𝙘𝗅𝙘𝗜𝙘𝗹𝙘𝘐𝙘𝘭𝙘𝙄𝙘𝙡𝙘𝙸𝙘𝚕𝙘𝚰𝙘𝛪𝙘𝜤𝙘𝝞𝙘𝞘𝙘𝟏𝙘𝟙𝙘𝟣𝙘𝟭𝙘𝟷𝚌1𝚌I𝚌l𝚌|𝚌Ɩ𝚌ǀ𝚌ℐ𝚌ℑ𝚌ℓ𝚌Ⅰ𝚌ⅼ𝚌∣𝚌⏽𝚌Ｉ𝚌ｌ𝚌￨𝚌𝐈𝚌𝐥𝚌𝐼𝚌𝑙𝚌𝑰𝚌𝒍𝚌𝓁𝚌𝓘𝚌𝓵𝚌𝔩𝚌𝕀𝚌𝕝𝚌𝕴𝚌𝖑𝚌𝖨𝚌𝗅𝚌𝗜𝚌𝗹𝚌𝘐𝚌𝘭𝚌𝙄𝚌𝙡𝚌𝙸𝚌𝚕𝚌𝚰𝚌𝛪𝚌𝜤𝚌𝝞𝚌𝞘𝚌𝟏𝚌𝟙𝚌𝟣𝚌𝟭𝚌𝟷e℮ℯⅇꬲｅ𝐞𝑒𝒆𝓮𝔢𝕖𝖊𝖾𝗲𝘦𝙚𝚎fſẝꞙꬵ𝐟𝑓𝒇𝒻𝓯𝔣𝕗𝖋𝖿𝗳𝘧𝙛𝚏gƍɡᶃℊｇ𝐠𝑔𝒈𝓰𝔤𝕘𝖌𝗀𝗴𝘨𝙜𝚐hℎｈ𝐡𝒉𝒽𝓱𝔥𝕙𝖍𝗁𝗵𝘩𝙝𝚑iıɩɪ˛ℹⅈⅰ⍳ｉ𝐢𝑖𝒊𝒾𝓲𝔦𝕚𝖎𝗂𝗶𝘪𝙞𝚒𝚤𝛊𝜄𝜾𝝸𝞲jⅉｊ𝐣𝑗𝒋𝒿𝓳𝔧𝕛𝖏𝗃𝗷𝘫𝙟𝚓k𝐤𝑘𝒌𝓀𝓴𝔨𝕜𝖐𝗄𝗸𝘬𝙠𝚔1Il|ƖǀℐℑℓⅠⅼ∣⏽Ｉｌ￨𝐈𝐥𝐼𝑙𝑰𝒍𝓁𝓘𝓵𝔩𝕀𝕝𝕴𝖑𝖨𝗅𝗜𝗹𝘐𝘭𝙄𝙡𝙸𝚕𝚰𝛪𝜤𝝞𝞘𝟏𝟙𝟣𝟭𝟷mrnr𝐧r𝑛r𝒏r𝓃r𝓷r𝔫r𝕟r𝖓r𝗇r𝗻r𝘯r𝙣r𝚗ꭇnꭇ𝐧ꭇ𝑛ꭇ𝒏ꭇ𝓃ꭇ𝓷ꭇ𝔫ꭇ𝕟ꭇ𝖓ꭇ𝗇ꭇ𝗻ꭇ𝘯ꭇ𝙣ꭇ𝚗ꭈnꭈ𝐧ꭈ𝑛ꭈ𝒏ꭈ𝓃ꭈ𝓷ꭈ𝔫ꭈ𝕟ꭈ𝖓ꭈ𝗇ꭈ𝗻ꭈ𝘯ꭈ𝙣ꭈ𝚗𝐫n𝐫𝐧𝐫𝑛𝐫𝒏𝐫𝓃𝐫𝓷𝐫𝔫𝐫𝕟𝐫𝖓𝐫𝗇𝐫𝗻𝐫𝘯𝐫𝙣𝐫𝚗𝑟n𝑟𝐧𝑟𝑛𝑟𝒏𝑟𝓃𝑟𝓷𝑟𝔫𝑟𝕟𝑟𝖓𝑟𝗇𝑟𝗻𝑟𝘯𝑟𝙣𝑟𝚗𝒓n𝒓𝐧𝒓𝑛𝒓𝒏𝒓𝓃𝒓𝓷𝒓𝔫𝒓𝕟𝒓𝖓𝒓𝗇𝒓𝗻𝒓𝘯𝒓𝙣𝒓𝚗𝓇n𝓇𝐧𝓇𝑛𝓇𝒏𝓇𝓃𝓇𝓷𝓇𝔫𝓇𝕟𝓇𝖓𝓇𝗇𝓇𝗻𝓇𝘯𝓇𝙣𝓇𝚗𝓻n𝓻𝐧𝓻𝑛𝓻𝒏𝓻𝓃𝓻𝓷𝓻𝔫𝓻𝕟𝓻𝖓𝓻𝗇𝓻𝗻𝓻𝘯𝓻𝙣𝓻𝚗𝔯n𝔯𝐧𝔯𝑛𝔯𝒏𝔯𝓃𝔯𝓷𝔯𝔫𝔯𝕟𝔯𝖓𝔯𝗇𝔯𝗻𝔯𝘯𝔯𝙣𝔯𝚗𝕣n𝕣𝐧𝕣𝑛𝕣𝒏𝕣𝓃𝕣𝓷𝕣𝔫𝕣𝕟𝕣𝖓𝕣𝗇𝕣𝗻𝕣𝘯𝕣𝙣𝕣𝚗𝖗n𝖗𝐧𝖗𝑛𝖗𝒏𝖗𝓃𝖗𝓷𝖗𝔫𝖗𝕟𝖗𝖓𝖗𝗇𝖗𝗻𝖗𝘯𝖗𝙣𝖗𝚗𝗋n𝗋𝐧𝗋𝑛𝗋𝒏𝗋𝓃𝗋𝓷𝗋𝔫𝗋𝕟𝗋𝖓𝗋𝗇𝗋𝗻𝗋𝘯𝗋𝙣𝗋𝚗𝗿n𝗿𝐧𝗿𝑛𝗿𝒏𝗿𝓃𝗿𝓷𝗿𝔫𝗿𝕟𝗿𝖓𝗿𝗇𝗿𝗻𝗿𝘯𝗿𝙣𝗿𝚗𝘳n𝘳𝐧𝘳𝑛𝘳𝒏𝘳𝓃𝘳𝓷𝘳𝔫𝘳𝕟𝘳𝖓𝘳𝗇𝘳𝗻𝘳𝘯𝘳𝙣𝘳𝚗𝙧n𝙧𝐧𝙧𝑛𝙧𝒏𝙧𝓃𝙧𝓷𝙧𝔫𝙧𝕟𝙧𝖓𝙧𝗇𝙧𝗻𝙧𝘯𝙧𝙣𝙧𝚗𝚛n𝚛𝐧𝚛𝑛𝚛𝒏𝚛𝓃𝚛𝓷𝚛𝔫𝚛𝕟𝚛𝖓𝚛𝗇𝚛𝗻𝚛𝘯𝚛𝙣𝚛𝚗n𝐧𝑛𝒏𝓃𝓷𝔫𝕟𝖓𝗇𝗻𝘯𝙣𝚗oᴏᴑℴꬽｏ𝐨𝑜𝒐𝓸𝔬𝕠𝖔𝗈𝗼𝘰𝙤𝚘𝛐𝛔𝜊𝜎𝝄𝝈𝝾𝞂𝞸𝞼p⍴ｐ𝐩𝑝𝒑𝓅𝓹𝔭𝕡𝖕𝗉𝗽𝘱𝙥𝚙𝛒𝛠𝜌𝜚𝝆𝝔𝞀𝞎𝞺𝟈q𝐪𝑞𝒒𝓆𝓺𝔮𝕢𝖖𝗊𝗾𝘲𝙦𝚚rꭇꭈ𝐫𝑟𝒓𝓇𝓻𝔯𝕣𝖗𝗋𝗿𝘳𝙧𝚛sƽꜱｓ𝐬𝑠𝒔𝓈𝓼𝔰𝕤𝖘𝗌𝘀𝘴𝙨𝚜t𝐭𝑡𝒕𝓉𝓽𝔱𝕥𝖙𝗍𝘁𝘵𝙩𝚝uʋᴜꞟꭎꭒ𝐮𝑢𝒖𝓊𝓾𝔲𝕦𝖚𝗎𝘂𝘶𝙪𝚞𝛖𝜐𝝊𝞄𝞾vᴠⅴ∨⋁ｖ𝐯𝑣𝒗𝓋𝓿𝔳𝕧𝖛𝗏𝘃𝘷𝙫𝚟𝛎𝜈𝝂𝝼𝞶wɯᴡ𝐰𝑤𝒘𝓌𝔀𝔴𝕨𝖜𝗐𝘄𝘸𝙬𝚠vvvᴠvⅴv∨v⋁vｖv𝐯v𝑣v𝒗v𝓋v𝓿v𝔳v𝕧v𝖛v𝗏v𝘃v𝘷v𝙫v𝚟v𝛎v𝜈v𝝂v𝝼v𝞶ᴠvᴠᴠᴠⅴᴠ∨ᴠ⋁ᴠｖᴠ𝐯ᴠ𝑣ᴠ𝒗ᴠ𝓋ᴠ𝓿ᴠ𝔳ᴠ𝕧ᴠ𝖛ᴠ𝗏ᴠ𝘃ᴠ𝘷ᴠ𝙫ᴠ𝚟ᴠ𝛎ᴠ𝜈ᴠ𝝂ᴠ𝝼ᴠ𝞶ⅴvⅴᴠⅴⅴⅴ∨ⅴ⋁ⅴｖⅴ𝐯ⅴ𝑣ⅴ𝒗ⅴ𝓋ⅴ𝓿ⅴ𝔳ⅴ𝕧ⅴ𝖛ⅴ𝗏ⅴ𝘃ⅴ𝘷ⅴ𝙫ⅴ𝚟ⅴ𝛎ⅴ𝜈ⅴ𝝂ⅴ𝝼ⅴ𝞶∨v∨ᴠ∨ⅴ∨∨∨⋁∨ｖ∨𝐯∨𝑣∨𝒗∨𝓋∨𝓿∨𝔳∨𝕧∨𝖛∨𝗏∨𝘃∨𝘷∨𝙫∨𝚟∨𝛎∨𝜈∨𝝂∨𝝼∨𝞶⋁v⋁ᴠ⋁ⅴ⋁∨⋁⋁⋁ｖ⋁𝐯⋁𝑣⋁𝒗⋁𝓋⋁𝓿⋁𝔳⋁𝕧⋁𝖛⋁𝗏⋁𝘃⋁𝘷⋁𝙫⋁𝚟⋁𝛎⋁𝜈⋁𝝂⋁𝝼⋁𝞶ｖvｖᴠｖⅴｖ∨ｖ⋁ｖｖｖ𝐯ｖ𝑣ｖ𝒗ｖ𝓋ｖ𝓿ｖ𝔳ｖ𝕧ｖ𝖛ｖ𝗏ｖ𝘃ｖ𝘷ｖ𝙫ｖ𝚟ｖ𝛎ｖ𝜈ｖ𝝂ｖ𝝼ｖ𝞶𝐯v𝐯ᴠ𝐯ⅴ𝐯∨𝐯⋁𝐯ｖ𝐯𝐯𝐯𝑣𝐯𝒗𝐯𝓋𝐯𝓿𝐯𝔳𝐯𝕧𝐯𝖛𝐯𝗏𝐯𝘃𝐯𝘷𝐯𝙫𝐯𝚟𝐯𝛎𝐯𝜈𝐯𝝂𝐯𝝼𝐯𝞶𝑣v𝑣ᴠ𝑣ⅴ𝑣∨𝑣⋁𝑣ｖ𝑣𝐯𝑣𝑣𝑣𝒗𝑣𝓋𝑣𝓿𝑣𝔳𝑣𝕧𝑣𝖛𝑣𝗏𝑣𝘃𝑣𝘷𝑣𝙫𝑣𝚟𝑣𝛎𝑣𝜈𝑣𝝂𝑣𝝼𝑣𝞶𝒗v𝒗ᴠ𝒗ⅴ𝒗∨𝒗⋁𝒗ｖ𝒗𝐯𝒗𝑣𝒗𝒗𝒗𝓋𝒗𝓿𝒗𝔳𝒗𝕧𝒗𝖛𝒗𝗏𝒗𝘃𝒗𝘷𝒗𝙫𝒗𝚟𝒗𝛎𝒗𝜈𝒗𝝂𝒗𝝼𝒗𝞶𝓋v𝓋ᴠ𝓋ⅴ𝓋∨𝓋⋁𝓋ｖ𝓋𝐯𝓋𝑣𝓋𝒗𝓋𝓋𝓋𝓿𝓋𝔳𝓋𝕧𝓋𝖛𝓋𝗏𝓋𝘃𝓋𝘷𝓋𝙫𝓋𝚟𝓋𝛎𝓋𝜈𝓋𝝂𝓋𝝼𝓋𝞶𝓿v𝓿ᴠ𝓿ⅴ𝓿∨𝓿⋁𝓿ｖ𝓿𝐯𝓿𝑣𝓿𝒗𝓿𝓋𝓿𝓿𝓿𝔳𝓿𝕧𝓿𝖛𝓿𝗏𝓿𝘃𝓿𝘷𝓿𝙫𝓿𝚟𝓿𝛎𝓿𝜈𝓿𝝂𝓿𝝼𝓿𝞶𝔳v𝔳ᴠ𝔳ⅴ𝔳∨𝔳⋁𝔳ｖ𝔳𝐯𝔳𝑣𝔳𝒗𝔳𝓋𝔳𝓿𝔳𝔳𝔳𝕧𝔳𝖛𝔳𝗏𝔳𝘃𝔳𝘷𝔳𝙫𝔳𝚟𝔳𝛎𝔳𝜈𝔳𝝂𝔳𝝼𝔳𝞶𝕧v𝕧ᴠ𝕧ⅴ𝕧∨𝕧⋁𝕧ｖ𝕧𝐯𝕧𝑣𝕧𝒗𝕧𝓋𝕧𝓿𝕧𝔳𝕧𝕧𝕧𝖛𝕧𝗏𝕧𝘃𝕧𝘷𝕧𝙫𝕧𝚟𝕧𝛎𝕧𝜈𝕧𝝂𝕧𝝼𝕧𝞶𝖛v𝖛ᴠ𝖛ⅴ𝖛∨𝖛⋁𝖛ｖ𝖛𝐯𝖛𝑣𝖛𝒗𝖛𝓋𝖛𝓿𝖛𝔳𝖛𝕧𝖛𝖛𝖛𝗏𝖛𝘃𝖛𝘷𝖛𝙫𝖛𝚟𝖛𝛎𝖛𝜈𝖛𝝂𝖛𝝼𝖛𝞶𝗏v𝗏ᴠ𝗏ⅴ𝗏∨𝗏⋁𝗏ｖ𝗏𝐯𝗏𝑣𝗏𝒗𝗏𝓋𝗏𝓿𝗏𝔳𝗏𝕧𝗏𝖛𝗏𝗏𝗏𝘃𝗏𝘷𝗏𝙫𝗏𝚟𝗏𝛎𝗏𝜈𝗏𝝂𝗏𝝼𝗏𝞶𝘃v𝘃ᴠ𝘃ⅴ𝘃∨𝘃⋁𝘃ｖ𝘃𝐯𝘃𝑣𝘃𝒗𝘃𝓋𝘃𝓿𝘃𝔳𝘃𝕧𝘃𝖛𝘃𝗏𝘃𝘃𝘃𝘷𝘃𝙫𝘃𝚟𝘃𝛎𝘃𝜈𝘃𝝂𝘃𝝼𝘃𝞶𝘷v𝘷ᴠ𝘷ⅴ𝘷∨𝘷⋁𝘷ｖ𝘷𝐯𝘷𝑣𝘷𝒗𝘷𝓋𝘷𝓿𝘷𝔳𝘷𝕧𝘷𝖛𝘷𝗏𝘷𝘃𝘷𝘷𝘷𝙫𝘷𝚟𝘷𝛎𝘷𝜈𝘷𝝂𝘷𝝼𝘷𝞶𝙫v𝙫ᴠ𝙫ⅴ𝙫∨𝙫⋁𝙫ｖ𝙫𝐯𝙫𝑣𝙫𝒗𝙫𝓋𝙫𝓿𝙫𝔳𝙫𝕧𝙫𝖛𝙫𝗏𝙫𝘃𝙫𝘷𝙫𝙫𝙫𝚟𝙫𝛎𝙫𝜈𝙫𝝂𝙫𝝼𝙫𝞶𝚟v𝚟ᴠ𝚟ⅴ𝚟∨𝚟⋁𝚟ｖ𝚟𝐯𝚟𝑣𝚟𝒗𝚟𝓋𝚟𝓿𝚟𝔳𝚟𝕧𝚟𝖛𝚟𝗏𝚟𝘃𝚟𝘷𝚟𝙫𝚟𝚟𝚟𝛎𝚟𝜈𝚟𝝂𝚟𝝼𝚟𝞶𝛎v𝛎ᴠ𝛎ⅴ𝛎∨𝛎⋁𝛎ｖ𝛎𝐯𝛎𝑣𝛎𝒗𝛎𝓋𝛎𝓿𝛎𝔳𝛎𝕧𝛎𝖛𝛎𝗏𝛎𝘃𝛎𝘷𝛎𝙫𝛎𝚟𝛎𝛎𝛎𝜈𝛎𝝂𝛎𝝼𝛎𝞶𝜈v𝜈ᴠ𝜈ⅴ𝜈∨𝜈⋁𝜈ｖ𝜈𝐯𝜈𝑣𝜈𝒗𝜈𝓋𝜈𝓿𝜈𝔳𝜈𝕧𝜈𝖛𝜈𝗏𝜈𝘃𝜈𝘷𝜈𝙫𝜈𝚟𝜈𝛎𝜈𝜈𝜈𝝂𝜈𝝼𝜈𝞶𝝂v𝝂ᴠ𝝂ⅴ𝝂∨𝝂⋁𝝂ｖ𝝂𝐯𝝂𝑣𝝂𝒗𝝂𝓋𝝂𝓿𝝂𝔳𝝂𝕧𝝂𝖛𝝂𝗏𝝂𝘃𝝂𝘷𝝂𝙫𝝂𝚟𝝂𝛎𝝂𝜈𝝂𝝂𝝂𝝼𝝂𝞶𝝼v𝝼ᴠ𝝼ⅴ𝝼∨𝝼⋁𝝼ｖ𝝼𝐯𝝼𝑣𝝼𝒗𝝼𝓋𝝼𝓿𝝼𝔳𝝼𝕧𝝼𝖛𝝼𝗏𝝼𝘃𝝼𝘷𝝼𝙫𝝼𝚟𝝼𝛎𝝼𝜈𝝼𝝂𝝼𝝼𝝼𝞶𝞶v𝞶ᴠ𝞶ⅴ𝞶∨𝞶⋁𝞶ｖ𝞶𝐯𝞶𝑣𝞶𝒗𝞶𝓋𝞶𝓿𝞶𝔳𝞶𝕧𝞶𝖛𝞶𝗏𝞶𝘃𝞶𝘷𝞶𝙫𝞶𝚟𝞶𝛎𝞶𝜈𝞶𝝂𝞶𝝼𝞶𝞶x×ⅹ⤫⤬⨯ｘ𝐱𝑥𝒙𝓍𝔁𝔵𝕩𝖝𝗑𝘅𝘹𝙭𝚡yɣʏᶌỿℽꭚｙ𝐲𝑦𝒚𝓎𝔂𝔶𝕪𝖞𝗒𝘆𝘺𝙮𝚢𝛄𝛾𝜸𝝲𝞬zᴢ𝐳𝑧𝒛𝓏𝔃𝔷𝕫𝖟𝗓𝘇𝘻𝙯𝚣'

In [ ]:
# s1="КΥá ⅼ ｈÀi??  \/\/"
# s3='Ｈⅰ Нοѡ aᏒÈ ｙoμ?'
# font_map = {'COMMON' : '../input/arialttf-font/arial.ttf', 'COPTIC': '../input/coptic-eye/Coptic Eyes Latin.ttf', 'LATIN':'../input/times-new-roman/Times New Roman/times new roman.ttf'}

# # Create black mask using Numpy and convert from BGR (OpenCV) to RGB (PIL)
# # image = cv2.imread('1.png') # If you were using an actual image
# image = np.zeros((100, 1000, 3), dtype=np.uint8)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# pil_image = Image.fromarray(image)

# # Draw non-ascii text onto image
# space=0
# for char in s1:
#     font = ImageFont.truetype(r'../input/unifont/unifont-15.0.01.ttf', 35)
#     draw = ImageDraw.Draw(pil_image)
#     draw.text((30+space, 30), char, font=font)
#     space+=10


# # Convert back to Numpy array and switch back from RGB to BGR
# img= np.asarray(pil_image)
# img= cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

# pil_image

In [ ]:
# s1="КΥá ⅼ ｈÀi??  \/\/"
s1="К"
s3='Ｈⅰ Нοѡ aᏒÈ ｙoμ?'
font_map = {'COMMON' : '../input/arialttf-font/arial.ttf', 'COPTIC': '../input/coptic-eye/Coptic Eyes Latin.ttf', 'LATIN':'../input/times-new-roman/Times New Roman/times new roman.ttf'}

# Create black mask using Numpy and convert from BGR (OpenCV) to RGB (PIL)
# image = cv2.imread('1.png') # If you were using an actual image
image = np.zeros((100, 1000, 3), dtype=np.uint8)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
pil_image = Image.fromarray(image)

# Draw non-ascii text onto image
space=0
for char in s1:
    font = ImageFont.truetype(r'../input/unifont/unifont-15.0.01.ttf', 35)
    draw = ImageDraw.Draw(pil_image)
    draw.text((30+space, 30), char, font=font)
    space+=10


# Convert back to Numpy array and switch back from RGB to BGR
img= np.asarray(pil_image)
img= cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

kernel = np.ones((2, 2), np.uint8)
img = cv2.dilate(img, kernel, iterations=1)
# img = cv2.erode(img, kernel, iterations=1)

img = cv2.threshold(cv2.medianBlur(img, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

plt.figure(figsize = (20,2))

plt.imshow(img, cmap='gray')

In [ ]:
text = pytesseract.image_to_string(img, config = "-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz --psm 6")
print(text[:-2]) ## remove unseen character

In [ ]:
hg.Categories.detect('𝐀')

# with CNN

In [ ]:
s1="|"
font_map = {'COMMON' : '../input/arialttf-font/arial.ttf', 'COPTIC': '../input/coptic-eye/Coptic Eyes Latin.ttf', 'LATIN':'../input/times-new-roman/Times New Roman/times new roman.ttf'}

# Create black mask using Numpy and convert from BGR (OpenCV) to RGB (PIL)
# image = cv2.imread('1.png') # If you were using an actual image
image = np.zeros((28, 28, 3), dtype=np.uint8)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
pil_image = Image.fromarray(image)

# Draw non-ascii text onto image
space=0
for char in s1:
    font = ImageFont.truetype(r'../input/unifont/unifont-15.0.01.ttf', 25)
    draw = ImageDraw.Draw(pil_image)
    draw.text((0+space, 0), char, font=font)
    space+=10


# Convert back to Numpy array and switch back from RGB to BGR
img= np.asarray(pil_image)
img= cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

kernel = np.ones((2, 2), np.uint8)
img = cv2.dilate(img, kernel, iterations=1)
# img = cv2.erode(img, kernel, iterations=1)

img = cv2.threshold(cv2.medianBlur(img, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# plt.figure(figsize = (20,2))

# plt.imshow(img, cmap='gray')

img = Image.fromarray(img)
img

In [ ]:
transform_tensor = transforms.ToTensor()
inp_img = transform_tensor(img)
inp_img = inp_img.unsqueeze(-1).permute(0, 3, 1, 2)

# From: [batch_size, height, width, channels]
# To: [batch_size, channels, depth, height, width]
# inp_image.permute(0, 3, 1, 2)

In [ ]:
inp_img.shape

In [ ]:
model = EMNIST_CNN(40, 160, 200, 0.4, 52).to(device)


checkpoint_path = r'/kaggle/input/hemnistcheckpoint/HEMNIST_CNN_20.pth'
# checkpoint_path = r'/kaggle/input/emnist-cnn-7/EMNIST_CNN_7.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

In [ ]:
idx_char[torch.argmax(model(inp_img)).item()]

# Export

In [ ]:
import shutil
shutil.make_archive('tesseract_ocr_results_uncased', 'zip', './tesseract_ocr_results')

In [ ]:
import shutil
shutil.make_archive('/kaggle/working/all_fonts', 'zip', '/kaggle/working/all_fonts')